In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time

path = "C:/R/chromedriver"
source_url = "https://map.kakao.com/"
driver = webdriver.Chrome(path)
driver.get(source_url) 
# 검색창
searchbox = driver.find_element_by_xpath("//*[@id='search.keyword.query']") 
# // input 가장 처음 input 찾기 , @ 속성표시
searchbox.send_keys("강남역 고기집")
searchbutton = driver.find_element_by_xpath("//*[@id='search.keyword.submit']")

driver.execute_script("arguments[0].click();", searchbutton)
time.sleep(1)

html = driver.page_source

InvalidSelectorException: Message: invalid selector: Unable to locate an element with the xpath expression //[@id='search.keyword.query'] because of the following error:
SyntaxError: Failed to execute 'evaluate' on 'Document': The string '//[@id='search.keyword.query']' is not a valid XPath expression.
  (Session info: chrome=92.0.4515.107)


In [11]:

soup = BeautifulSoup(html, "html.parser")
# 페이지 url 
moreviews = soup.find_all(name = "a", attrs = {"class":"moreview"})
page_urls = []
for moreview in moreviews :
    page_url = moreview.get("href")
    print(page_url)
    page_urls.append(page_url)
driver.close()

https://place.map.kakao.com/85570955
https://place.map.kakao.com/1503746075
https://place.map.kakao.com/95713992
https://place.map.kakao.com/741391811
https://place.map.kakao.com/2011092566
https://place.map.kakao.com/13573220
https://place.map.kakao.com/2062959414
https://place.map.kakao.com/1648266796
https://place.map.kakao.com/168079537
https://place.map.kakao.com/263830255
https://place.map.kakao.com/27238067
https://place.map.kakao.com/26431943
https://place.map.kakao.com/1780387311
https://place.map.kakao.com/1907052666
https://place.map.kakao.com/1052874675
https://place.map.kakao.com/1576421052


In [23]:
# for p in page_urls :
#     print(p)
columns = ['score','review']
df = pd.DataFrame(columns = columns)
driver = webdriver.Chrome(path)
for page in page_urls :
    driver.get(page)
    time.sleep(1.5)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    # 리뷰
    contents_div = soup.find(name = "div", attrs={"class":"evaluation_review"})
    # 평점
    rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
    # 리뷰
    reviews = contents_div.find_all(name = "p", attrs={"class":"txt_comment"})
    print(rates.text)
    for rate, review in zip(rates, reviews) :
        row = [rate.text[0], review.find(name="span").text]
        series = pd.Series(row, index=df.columns)
        df = df.append(series, ignore_index=True)
        
    for button_num in range(2, 6) :
        try :
            another_reviews = driver.find_element_by_xpath\
                ("//a[@data-page='"+str(button_num)+"']")
            another_reviews.click()
            time.sleep(1.5)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            
            contents_div = soup.find\
                (name="div", attrs={"class":"evaluation_reivew"})
            rates = contents_div.find_all\
                (name = "em", attrs = {"class":"num_rate"})
            raviews = contents_div.find_all\
                (name = "p", attrs = {"class":"txt_comment"})
            
            for rate, review in zip(rates, reviews) :
                row = [rate.text[0], review.find(name="span").text]
                series = pd.Series(row, index=df.columns)
                df = df.append(series, ignore_index=True)
        except :
            break
driver.close()

[<em class="num_rate">1<span class="screen_out">점</span></em>, <em class="num_rate">5<span class="screen_out">점</span></em>, <em class="num_rate">5<span class="screen_out">점</span></em>, <em class="num_rate">4<span class="screen_out">점</span></em>, <em class="num_rate">4<span class="screen_out">점</span></em>]
[<p class="txt_comment "><span>예약할 때는 룸을 주기로 하고 홀을 주고, 덥고, 직원들이 정신이 없어 그 가격에 내가 직접 구워먹고 갈비살, 등심은 질기고 냉면은 맛이 없고 장어 양념들도 제 때 안 가져다 주고 회식으로 한시간만에 120만원을 썼는데 이런 경험 처음입니다.</span><button class="btn_fold" type="button">더보기</button></p>, <p class="txt_comment "><span>점심식사 잘했던곳.후식커피한잔 하기도 좋고 주차가능합니다. 음식 맛있고 직원분 친절하여 절로 미소가 지어졌어요. </span><button class="btn_fold" type="button">더보기</button></p>, <p class="txt_comment "><span>新鮮でおいしいです。</span><button class="btn_fold" type="button">더보기</button></p>, <p class="txt_comment "><span>녹는다 녹아</span><button class="btn_fold" type="button">더보기</button></p>, <p class="txt_comment "><span></span><button class="btn_fold" type="button">더보기</button></p>]
[<em

[<em class="num_rate">2<span class="screen_out">점</span></em>, <em class="num_rate">5<span class="screen_out">점</span></em>, <em class="num_rate">2<span class="screen_out">점</span></em>, <em class="num_rate">5<span class="screen_out">점</span></em>, <em class="num_rate">5<span class="screen_out">점</span></em>]
[<p class="txt_comment "><span>친절하고 깨끗하지만 곱창이 너무 질겨서 실망이었어요ㅜㅜ 별점에비해 맛은 그닥..대파김치덕분에 다 먹었네요</span><button class="btn_fold" type="button">더보기</button></p>, <p class="txt_comment "><span>곱창을 좋아해서 생각없이들어가서먹었는데 곱도많이들어있고 냄새도안나고 질긴것도없고 맛있네요  볶음밥이 신의한수!!  가게내부도 깔끔하고 좋았어요</span><button class="btn_fold" type="button">더보기</button></p>, <p class="txt_comment "><span>그냥 평범함...ㄹㅇ</span><button class="btn_fold" type="button">더보기</button></p>, <p class="txt_comment "><span>대충 친구랑 밥 먹으러 들어갔다가 생각보다 맛있어서 괜찮았어요 사장님도 친절하시고 감사합니당</span><button class="btn_fold" type="button">더보기</button></p>, <p class="txt_comment "><span>어후 소주 안먹고는 안될 맛이었어요 술마실 생각 없었는데;; 진짜 맛있게 잘 먹고 배터질거같다 하면서 나오면서도 여기 또 오자 하고 나왔네요</spa

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   score   80 non-null     object
 1   review  80 non-null     object
dtypes: object(2)
memory usage: 1.4+ KB


In [25]:
df.head()

,score,review
0,1,"예약할 때는 룸을 주기로 하고 홀을 주고, 덥고, 직원들이 정신이 없어 그 가격에 ..."
1,5,점심식사 잘했던곳.후식커피한잔 하기도 좋고 주차가능합니다. 음식 맛있고 직원분 친절...
2,5,新鮮でおいしいです。
3,4,녹는다 녹아
4,4,


In [27]:
# 긍부정 평가
df['y'] = df['score'].apply(lambda x : 1 if float(x) > 3 else 0)
df

,score,review,y
0,1,"예약할 때는 룸을 주기로 하고 홀을 주고, 덥고, 직원들이 정신이 없어 그 가격에 ...",0
1,5,점심식사 잘했던곳.후식커피한잔 하기도 좋고 주차가능합니다. 음식 맛있고 직원분 친절...,1
2,5,新鮮でおいしいです。,1
3,4,녹는다 녹아,1
4,4,,1
...,...,...,...
75,2,이렇게 대기가 긴 맛집인줄 모르고 갔다가 엄청 기다림 예써라는 어플로 대기 하던데 ...,0
76,1,단짠의 정석. 진짜 정석으로 달고 짬. 질리는 맛. 사장님이랑 와이프로 추정되는 ...,0
77,4,만족스러움! 맛있어용,1
78,1,곱창은 없고 대창만 들어있어서 느끼한데 양념은 너무 매워서 위에 탈이나 고생했습니다ㅠㅠ,0


In [28]:
df.y.value_counts()

1    44
0    36
Name: y, dtype: int64

In [29]:
df.to_csv('review_data.csv', index=False)